In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import matplotlib.pyplot as plt
import os 
os.chdir("D:/Training/Academy/ML(Python)/Cases/Glass_Identification")
from sklearn.preprocessing import LabelEncoder 

In [2]:
glass = pd.read_csv("Glass.csv")
dum_gls = pd.get_dummies(glass)
X = dum_gls.iloc[:,:-6]
y = dum_gls.iloc[:,-6:]

In [3]:
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size = 0.3, 
                                                    random_state=25,stratify=glass['Type'])
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

In [4]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([149, 9])
torch.Size([149, 6])


In [5]:
type(X_torch)

torch.Tensor

In [6]:
X_scl_trn.shape[1]

9

### Model Definition

In [7]:
torch.manual_seed(25)
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1],out_features=18),
                      nn.ReLU(),
                      nn.Linear(in_features=18,out_features=6))
summary(model, (1,X_scl_trn.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 18]             180
              ReLU-2                [-1, 1, 18]               0
            Linear-3                 [-1, 1, 6]             114
Total params: 294
Trainable params: 294
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.05
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Generating predictions with default weights

In [9]:
y_pred = model(X_torch.float())
y_pred[:3]

tensor([[ 0.1255,  0.3045, -0.0804,  0.0280,  0.1467, -0.3462],
        [ 0.1773,  0.2718, -0.0696, -0.0141,  0.1395, -0.3197],
        [ 0.1468,  0.2995, -0.0838,  0.0472,  0.1369, -0.3467]],
       grad_fn=<SliceBackward0>)

In [10]:
print(y_torch.shape)

torch.Size([149, 6])


In [11]:
#y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([149, 6])
torch.Size([149, 6])


### Initial Log Loss

In [12]:
loss = criterion(y_pred, y_torch.float())
loss

tensor(1.7186, grad_fn=<DivBackward1>)

In [13]:
for epoch in np.arange(0,1000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())

       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       if epoch%100 == 0:
          print('epoch: ', epoch+1,' loss: ', loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  1.7186344861984253
epoch:  101  loss:  1.46933913230896
epoch:  201  loss:  1.3758622407913208
epoch:  301  loss:  1.2651137113571167
epoch:  401  loss:  1.1757513284683228
epoch:  501  loss:  1.1176071166992188
epoch:  601  loss:  1.075873851776123
epoch:  701  loss:  1.0433498620986938
epoch:  801  loss:  1.0167316198349
epoch:  901  loss:  0.9942196607589722


### Training Set Log Loss

In [14]:
loss

tensor(0.9749, grad_fn=<DivBackward1>)

In [15]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [16]:
lin_output = model(X_torch_test.float()) # Equivalent predict_proba / predict
type(lin_output)

torch.Tensor

In [17]:
y_wt_sum = model(X_torch_test.float()) 
softmax = nn.Softmax(dim=1)
pred_proba = softmax(y_wt_sum)
pred_proba

tensor([[5.8695e-01, 2.9242e-01, 7.6382e-03, 1.4663e-03, 1.1330e-02, 1.0019e-01],
        [4.1819e-01, 4.1192e-01, 2.3809e-02, 1.0379e-02, 3.0064e-02, 1.0564e-01],
        [4.4359e-01, 4.0753e-01, 1.8132e-02, 6.7682e-03, 2.3336e-02, 1.0064e-01],
        [4.3652e-01, 4.1361e-01, 1.8634e-02, 6.8582e-03, 2.3824e-02, 1.0055e-01],
        [4.2667e-01, 4.0766e-01, 2.2095e-02, 9.9252e-03, 2.8931e-02, 1.0472e-01],
        [6.0046e-01, 2.8183e-01, 7.6809e-03, 2.1972e-03, 1.2444e-02, 9.5389e-02],
        [5.7995e-01, 2.9570e-01, 8.8843e-03, 2.1145e-03, 1.3245e-02, 1.0011e-01],
        [4.9613e-01, 3.5449e-01, 1.5490e-02, 6.2269e-03, 2.2830e-02, 1.0483e-01],
        [4.5836e-01, 3.9298e-01, 1.7192e-02, 6.7154e-03, 2.3222e-02, 1.0153e-01],
        [3.6741e-01, 4.3580e-01, 3.2678e-02, 1.6015e-02, 3.9746e-02, 1.0835e-01],
        [1.2470e-01, 3.4742e-01, 1.0861e-01, 2.3353e-01, 1.1198e-01, 7.3766e-02],
        [4.9983e-01, 3.6921e-01, 1.2577e-02, 3.7587e-03, 1.7100e-02, 9.7525e-02],
        [3.7520e

`detach().numpy()` converts `torch.Tensor` into `numpy` object

In [18]:
y_pred_prob = pred_proba.detach().numpy()
y_pred_prob.shape

(65, 6)

In [19]:
y_pred = np.argmax(pred_proba.detach().numpy(), axis=1 )
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 3, 3, 1, 3, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 3, 0, 1, 1, 3, 1, 1, 0, 0, 3, 3, 1, 0, 0, 0, 0,
       3, 1, 1, 1, 0, 3, 1, 1, 0, 0, 1, 1, 3, 3, 0, 0, 3, 1, 1, 0, 1])

### Test Set Accuracy Score

In [20]:
y_test_org = np.argmax(y_test, axis=1 )
print(accuracy_score(y_test_org,y_pred))

0.49230769230769234


### Test Set Log Loss

In [21]:
log_loss(y_test_org, y_pred_prob)

1.0940806430203334